In [5]:
#importing required libraries
import cv2
import dlib
from imutils import face_utils
from pygame import mixer
import threading
import datetime
import math

pygame 2.5.0 (SDL 2.28.0, Python 3.10.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [6]:
#Function to find euclidean distance
def euclidean_distance(a, b):
    if len(a) != len(b):
        raise ValueError("Points should have the same number of dimensions.")
    
    squared_distance = sum((x - y) ** 2 for x, y in zip(a, b))
    return math.sqrt(squared_distance)

# Function to calculate eye aspect ratio
def eye_aspect_ratio(eye):
    A = euclidean_distance(eye[1], eye[5])
    B = euclidean_distance(eye[2], eye[4])
    C = euclidean_distance(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

# Function to calculate mouth aspect ratio
def mouth_aspect_ratio(mouth):
    A = euclidean_distance(mouth[2], mouth[10])
    B = euclidean_distance(mouth[4], mouth[8])
    C = euclidean_distance(mouth[0], mouth[6])
    mar = (A + B) / (2.0 * C)
    return mar



In [7]:
# Function to detect drowsiness and record video
def detect_drowsiness(record_video, show_landmarks):
    global detection_running

    shape_predictor_path = "shape_predictor_68_face_landmarks.dat"

    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error opening webcam.")
        return

    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor(shape_predictor_path)

    mixer.init()
    sound = mixer.Sound('alarm.wav')

    sleep_frames = 0
    active_frames = 0
    alarm_on = False
    drowsy_count = 0
    drowsy_timestamps = []

    # Video recording setup
    if record_video:
        frame_width = int(cap.get(3))
        frame_height = int(cap.get(4))
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter('output.mp4', fourcc, 20.0, (frame_width, frame_height))

    while detection_running:
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = detector(gray)

        for face in faces:
            landmarks = predictor(gray, face)
            landmarks = face_utils.shape_to_np(landmarks)

            left_eye = landmarks[36:42]
            right_eye = landmarks[42:48]
            mouth = landmarks[48:68]

            left_ear = eye_aspect_ratio(left_eye)
            right_ear = eye_aspect_ratio(right_eye)
            avg_ear = (left_ear + right_ear) / 2.0
            mar = mouth_aspect_ratio(mouth)

            if avg_ear < 0.25:
                sleep_frames += 1
                active_frames = 0

                if sleep_frames >= 20:
                    if not alarm_on:
                        sound.play()
                        alarm_on = True
                        drowsy_count += 1
                        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                        drowsy_timestamps.append(timestamp)

                    cv2.putText(frame, "ALERT: Drowsiness Detected", (200, 30),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            else:
                active_frames += 1
                sleep_frames = 0

                if active_frames >= 10:
                    if alarm_on:
                        sound.stop()
                        alarm_on = False
                cv2.putText(frame, "Active State", (200, 30),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)        

            # Detect yawning
            if mar > 0.5:
                cv2.putText(frame, "ALERT: Yawning", (200, 60),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

            cv2.putText(frame, f"EAR: {avg_ear:.2f}", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            cv2.putText(frame, f"MAR: {mar:.2f}", (10, 60),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

            # Show landmarks on the frame
            if show_landmarks:
                for (x, y) in landmarks:
                    cv2.circle(frame, (x, y), 1, (255, 255, 255), -1)

        cv2.imshow("Drowsiness Detection", frame)

        if record_video and detection_running:
            out.write(frame)

        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

    # Save the report to "reports.txt"
    with open("reports.txt", "w") as file:
        file.write(f"Total Drowsy Occurrences: {drowsy_count}\n")
        file.write("Drowsy Timestamps:\n")
        for timestamp in drowsy_timestamps:
            file.write(f"- {timestamp}\n")

    if record_video:
        out.release()

    print("Drowsiness detection completed. Report created.")



In [8]:
# Function to start drowsiness detection and video recording
def start_detection():
    global detection_running
    record_video = record_video_var.get()
    show_landmarks = show_landmarks_var.get()
    if not detection_running:
        detection_running = True
        thread = threading.Thread(target=detect_drowsiness, args=(record_video, show_landmarks))
        thread.start()

# Function to stop drowsiness detection
def stop_detection():
    global detection_running
    detection_running = False



In [9]:
# Function to show reports
def show_reports():
    try:
        with open("reports.txt", "r") as file:
            reports = file.read()

        # Create a new window to display the reports
        report_window = tk.Toplevel(window)
        report_window.title("Drowsiness Detection Reports")

        # Create a text widget to display the reports
        report_text = tk.Text(report_window, wrap=tk.WORD, font=("Arial", 14), bg="white")
        report_text.insert(tk.END, reports)
        report_text.pack(fill=tk.BOTH, expand=True)

    except FileNotFoundError:
        print("Reports file not found.")
    except Exception as e:
        print("Error occurred while reading the reports:", e)



In [10]:
# Function to show recorded videos using OpenCV
def show_recorded_videos():
    try:
        cap = cv2.VideoCapture('output.mp4')
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            cv2.imshow('Recorded Video', frame)
            if cv2.waitKey(25) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()

    except FileNotFoundError:
        print("Recorded video not found.")
    except Exception as e:
        print("Error occurred while showing the recorded video:", e)